<a href="https://colab.research.google.com/github/MpMan123/Hallucination_Detection-Legendary-and-Mighty-Capybara/blob/main/notebooks/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!git clone https://github.com/MpMan123/Hallucination_Detection-Legendary-and-Mighty-Capybara.git
%cd Hallucination_Detection-Legendary-and-Mighty-Capybara
!git branch -a
!git checkout remotes/origin/LanChiBranch
!git branch

Cloning into 'Hallucination_Detection-Legendary-and-Mighty-Capybara'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 61 (delta 13), reused 31 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 5.87 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/Hallucination_Detection-Legendary-and-Mighty-Capybara
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/LanChiBranch
  remotes/origin/PhuocBranch
  remotes/origin/QuanBranch
  remotes/origin/main
  remotes/origin/origin/PhuocBranch
Note: switching to 'remotes/origin/LanChiBranch'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new bran

In [27]:

import os
!ls
!ls /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/

with open("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/config_bert_base_multilingual.json", "r") as file:
  config = json.load(file)
print(config["model_name"])
print(config["batch_size"])
print(config["learning_rate"])
print(config["output_dir"])
print(config["log_dir"])


baseline.ipynb
config_bert_base_multilingual.json  config_model_unknown.json
bert-base-multilingual-cased
16
2e-05
results/checkpoints
results/logs/


In [34]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00


In [74]:
from transformers import (
                TrainingArguments,
                AutoModelForSequenceClassification,
                AutoTokenizer,
                Trainer,
                DataCollatorWithPadding)
import os
import json
import evaluate
from base_model import load_model
from datasets import load_dataset

!git branch
!ls
# Loading configuration
with open("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/config_bert_base_multilingual.json", "r") as file:
  config = json.load(file)
print(config["model_name"])
MODEL_NAME = config["model_name"]
BATCH_SIZE = config["batch_size"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]
OUTPUT_DIR = config["output_dir"]
LOG_DIR = config["log_dir"]

# Load model
model = load_model(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# %cd /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed

# Load dataset
dataset = load_dataset("csv", data_files={
    "train":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/train.csv",
    "evaluate":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/final_input.csv"
})
print(dataset)
print(dataset["train"][:2])



# def preprocess(examples):
#     text = examples["context"] + " " + examples["prompt"] + " " + examples["answer"]
#     return tokenizer(text, truncation=True, max_length=256)

# encoded_dataset = dataset.map(preprocess, batched=True)
# encoded_dataset = encoded_dataset.remove_columns(["id", "context", "prompt", "answer"])
# encoded_dataset = encoded_dataset.rename_column("label", "labels")
# encoded_dataset.set_format("torch")

# # Metric functions
# accuracy = evaluate.load("accuracy")
# f1 = evaluate.load("f1")

# def calculate_metrics(p):
#     preds = p.predictions.argmax(axis=-1)
#     acc = accuracy.compute(predictions=preds, references=p.label_ids)
#     f1_score = f1.compute(predictions=preds, references=p.label_ids, average="macro")
#     return {"accuracy": acc["accuracy"], "f1_macro": f1_score["f1"]}

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# # Training args
# training_args = TrainingArguments(
#     output_dir=OUTPUT_DIR,      # thư mục lưu checkpoint
#     evaluation_strategy="epoch",           # evaluate sau mỗi epoch
#     save_strategy="epoch",                 # save checkpoint sau mỗi epoch
#     save_total_limit=2,                    # (tùy chọn) chỉ giữ 2 checkpoint gần nhất
#     load_best_model_at_end=True,           # load model tốt nhất sau khi train
#     metric_for_best_model="f1_macro",      # chọn best dựa trên macro-F1
#     greater_is_better=True,                # F1 càng cao càng tốt
#     num_train_epochs= NUM_EPOCHS,                    # số epoch train
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=BATCH_SIZE,
#     learning_rate=LEARNING_RATE,
#     weight_decay=0.01,
#     logging_dir=LOG_DIR,            # log để theo dõi bằng TensorBoard
#     logging_steps=50                       # in log mỗi 50 step
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["evaluate"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=calculate_metrics
# )

# os.makedirs(OUTPUT_DIR, exist_ok=True)
# os.makedirs(LOG_DIR, exist_ok=True)

# if __name__ == "__main__":
#     trainer.train()
#     trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
#     tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))

* (HEAD detached at origin/LanChiBranch)
  main
clean_data.csv	 final_input.csv   pipeline.py	     src
clean_data.xlsx  final_input.xlsx  preprocess.py     vietnamese-stopwords.txt
data		 main.py	   README.md	     vihallu-public-test.csv
eda.py		 notebooks	   requirements.txt
bert-base-multilingual-cased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: Unable to find '/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/final_input.csv'

In [108]:
from transformers import (
                TrainingArguments,
                AutoModelForSequenceClassification,
                AutoTokenizer,
                Trainer,
                DataCollatorWithPadding)
import os
import json
import evaluate
from base_model import load_model
from datasets import load_dataset

!git pull origin main
!ls /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/

# Loading configuration
with open("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/config_bert_base_multilingual.json", "r") as file:
  config = json.load(file)
MODEL_NAME = config["model_name"]
BATCH_SIZE = config["batch_size"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]
OUTPUT_DIR = config["output_dir"]
LOG_DIR = config["log_dir"]

# Load model
model = load_model(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# %cd /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed

# Load dataset
dataset = load_dataset("csv", data_files={
    "train":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/train.csv",
})
hello = load_dataset("csv", data_files={
    "evaluate":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/evaluate.csv"
})
print(dataset)
print(hello)

From https://github.com/MpMan123/Hallucination_Detection-Legendary-and-Mighty-Capybara
 * branch            main       -> FETCH_HEAD
Already up to date.
evaluate.csv  train.csv


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating evaluate split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'prompt', 'response', 'label', 'clean_text', 'representation'],
        num_rows: 7000
    })
})
DatasetDict({
    evaluate: Dataset({
        features: ['id', 'context', 'prompt', 'response', 'predict_label', 'clean_text', 'representation'],
        num_rows: 1000
    })
})
